<a href="https://colab.research.google.com/github/Mattias36/binary_classification_cats_and_dogs/blob/main/cats_vs_dogs_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
dogs-vs-cats.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_dataset = datasets.ImageFolder('/content/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = datasets.ImageFolder('/content/test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 32 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [ ]:
# Modyfikacja 1: Dodanie trzeciej warstwy konwolucyjnej
class ModifiedCNN1(nn.Module):
    def __init__(self):
        super(ModifiedCNN1, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)  # Nowa warstwa konwolucyjna
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))  # Aktywacja dla nowej warstwy
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 64 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
class ModifiedCNN2(nn.Module):
    def __init__(self):
        super(ModifiedCNN2, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 32 * 32, 256)  # Więcej neuronów w warstwie w pełni połączonej
        self.fc2 = nn.Linear(256, 64)           # Dodanie dodatkowej warstwy w pełni połączonej
        self.fc3 = nn.Linear(64, 2)             # Wyjściowa warstwa

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 32 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))  # Nowa warstwa w pełni połączona
        x = self.fc3(x)
        return x

In [ ]:
import torch.optim as optim
import torch

# Funkcja do trenowania modelu
def train_model(model, train_loader, test_loader, epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")

    # Testowanie modelu
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total}%')


In [ ]:
# Trening oryginalnej sieci
print("Training SimpleCNN:")
model1 = SimpleCNN()
train_model(model1, train_loader, test_loader)

# Trening zmodyfikowanej sieci 1
print("\nTraining ModifiedCNN1:")
model2 = ModifiedCNN1()
train_model(model2, train_loader, test_loader)

# Trening zmodyfikowanej sieci 2
print("\nTraining ModifiedCNN2:")
model3 = ModifiedCNN2()
train_model(model3, train_loader, test_loader)


Training SimpleCNN:
Epoch 1, Loss: 0.648064333486557
Epoch 2, Loss: 0.5113909238338471
Epoch 3, Loss: 0.4313260902643204
Epoch 4, Loss: 0.35419988745450975
Epoch 5, Loss: 0.26575555056333544
Epoch 6, Loss: 0.16709813506305218
Epoch 7, Loss: 0.09789655162990094
Epoch 8, Loss: 0.055150156746432186
Epoch 9, Loss: 0.03539622383601963
Epoch 10, Loss: 0.03432973359450698
Accuracy: 76.8%

Training ModifiedCNN1:
Epoch 1, Loss: 0.576268492937088
Epoch 2, Loss: 0.44448801820278167
Epoch 3, Loss: 0.37995197861194613
Epoch 4, Loss: 0.3153878596663475
Epoch 5, Loss: 0.24261016152501105
Epoch 6, Loss: 0.16146811110973358
Epoch 7, Loss: 0.09728029003590345
Epoch 8, Loss: 0.06720549902692437
Epoch 9, Loss: 0.04803343488953542
Epoch 10, Loss: 0.042323705410026016
Accuracy: 83.1%

Training ModifiedCNN2:
Epoch 1, Loss: 0.5867571165084838
Epoch 2, Loss: 0.4516099758148193
Epoch 3, Loss: 0.3397676756620407
Epoch 4, Loss: 0.18473723510205745
Epoch 5, Loss: 0.06759610903896392
Epoch 6, Loss: 0.03413019123352